# Unpack Packages

In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
from MSD_utils import get_data_pups, build_time_array, return_average, avg_all, graph_single_variable
from MSD_utils import SD_all, return_SD, range_and_ticks, choose_y_axis_params, data_prep_for_plotting_pups
from MSD_utils import fill_in_and_split, plot_traj_length_histogram, plot_traj, filter_out_short_traj
from MSD_utils import plot_trajectory_overlay, quality_control

from MSD_utils import diffusion_coefficient_point_derivative, diffusion_coefficient_linear_regression
from MSD_utils import calculate_diffusion_coefficients, diffusion_bar_chart, summary_barcharts
from MSD_utils import calculate_MMSDs, plot_general_histogram, plot_MSD_histogram, plot_all_MSD_histograms
from MSD_utils import fillin2, MSD_iteration, vectorized_MMSD_calcs

In [7]:
folder = "./{genotype}/{pup}/{slic}/"
path = "./{genotype}/{pup}/{slic}/geoM2xy_{sample_name}.csv"
frames = 480
SD_frames = [1, 5, 10, 20, 30]
conversion = (0.31, 9.89, 1)#(0.3, 3.95, 1)
to_frame = 479
dimension = "2D"
time_to_calculate = 1

base_name = "WT"
test_bins = np.linspace(0, 75, 76)

# name = 'RED_KO_PEG_P1_S1_cortex'
cut = 4
totvids = 15
frame_m = 480  # atm I can't go lower than the actual value.
conversion = (0.31, 9.89, 1)

parameters = {}
parameters["channels"] = ["RED"]
parameters["genotypes"] = ["WT"]
parameters["pups"] = ["P195", "P199", "P200", "P202"]
parameters["surface functionalities"] = ["PEG"]
parameters["slices"] = ["S1", "S2", "S3", "S4", "S5"]
parameters["regions"] = ["cortex"]
parameters["replicates"] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
parameters["slice suffixes"] = ['a', 'b', 'c', 'd', 'e']


channels = parameters["channels"]
genotypes = parameters["genotypes"]
pups = parameters["pups"]
surface_functionalities = parameters["surface functionalities"]
slices = parameters["slices"]
regions = parameters["regions"]
replicates = parameters["replicates"]
suffixes = parameters["slice suffixes"]

In [ ]:
geoM1x = {}
geoM1y = {}
geoM2xy = {}
SM1x = {}
SM1y = {}
SM2xy = {}

for channel in channels:
    for genotype in genotypes:
        for surface_functionality in surface_functionalities:
            for region in regions:
                for pup in pups:
                    slice_counter = 0
                    for slic in slices:
                        suffix = suffixes[slice_counter]
                        sample_name = "{}_{}_{}_{}".format(channel, surface_functionality, pup, slic)
                        DIR = folder.format(genotype=genotype, pup=pup, slic=slic)
                        total1, xs, ys, x, y = MSD_iteration(DIR, sample_name, cut, totvids, conversion, frame_m)
                        
                        geoM1x[sample_name], geoM1y[sample_name], geoM2xy[sample_name], SM1x[sample_name], SM1y[sample_name],\
                            SM2xy[sample_name] = vectorized_MMSD_calcs(frames, total1, xs, ys, x, y, frame_m)
                        np.savetxt(DIR+'geoM2xy_{}.csv'.format(sample_name), geoM2xy[sample_name], delimiter=',')

                        slice_counter = slice_counter + 1

In [ ]:
path

In [ ]:
frames = 480

In [8]:
data, avg_over_slices, avg_over_pups, names_with_replicates, time, time_SD, average_over_slices, average_over_pups,\
    all_SD_over_slices, all_SD_over_pups = data_prep_for_plotting_pups(path, frames, SD_frames, conversion, to_frame, parameters)

In [ ]:
plot_all_MSD_histograms(parameters, folder, SM2xy, time, test_bins, 1, set_y_limit=True, y_range=1000, set_x_limit=True, x_range=60)

In [13]:
y_range, ticks_y, dec_y, x_range, ticks_x, dec_x = 5.1, 0.5, 1, 2.6, 0.5, 1

In [26]:
all_SM_over_pups

{'RED_WT_PEG_cortex': array([ 0.26124644,  0.23469175,  0.34159912,  0.42423459,  0.42962517])}

In [25]:
all_SM_over_pups = {}
for keys in all_SD_over_pups:
    all_SM_over_pups[keys] = all_SD_over_pups[keys]/np.sqrt(5)

In [27]:
in_name1 = "WT"
in_name2 = "RED"

graph_single_variable(average_over_pups, all_SM_over_pups, time, time_SD, SD_frames, in_name1, in_name2, to_frame=to_frame, y_range=y_range,
                      ticks_y=ticks_y, dec_y=dec_y, x_range=x_range, ticks_x=ticks_x, dec_x=dec_x,
                      modify_labels=True, label_identifier="PEG_", x_manual=True, base_name=base_name)
plt.show()